RAIN - Real & Artificial Intelligence for Neuroscience

## Video handling

Welcome to my video handling notebook!

Here you'll find are a compilation of many functions I've written when working with videos.

Since we record behavioral experiments, sometimes it is useful to cut, reshape, draw on, and add text to videos, among other things.

In [68]:
import os
import numpy as np

import cv2
from tkinter import Tk, filedialog, messagebox

In [69]:
def select_video_files() -> list:
    """Select video files using a file dialog.

    Returns:
        list: List of selected video files.
    """
    import tkinter.filedialog as filedialog
    from tkinter import Tk

    # Initialize Tkinter and hide the root window
    root = Tk()
    root.withdraw()
    
    # Open file dialog to select video files
    video_files = filedialog.askopenfilenames(
        title="Select Video Files",
        filetypes=[("Video Files", "*.mp4 *.avi *.mkv *.mov")]
    )
    if not video_files:
        raise ValueError("No video files selected.")
    
    print(f"Selected {len(video_files)} videos.")

    return video_files

In [70]:
def merge_frames(video_files: list) -> np.ndarray:
    """
    Merge the first frame of each video file into a single image.

    Args:
        video_files (list): List of video files.
    
    Returns:
        np.ndarray: Merged image.
    """
    merged_image = None
    
    for video_file in video_files:
        cap = cv2.VideoCapture(video_file)
        success, frame = cap.read()
        cap.release()
        
        if not success:
            print(f"Could not read first frame of {video_file}")
            continue
        
        # Calculate transparency
        transparency = round(1 / len(video_files), 4)
        transparent_frame = (frame * transparency).astype(np.uint8)
        
        if merged_image is None:
            # Initialize merged image
            merged_image = np.zeros_like(transparent_frame)
        
        # Add transparent frame to the merged image
        merged_image = cv2.add(merged_image, transparent_frame)
    
    return merged_image

In [71]:
def define_rectangle(x1, y1, x2, y2):
                width = int(round(abs(x2 - x1)))
                height = int(round(abs(y2 - y1)))
                center = [int(round((x1+x2)//2)), int(round((y1+y2)//2))] # Round to integer
                return center, width, height

def draw_rectangle(image, center, width, height, angle = 0, color = (0, 255, 0), thickness = 2):
    box = cv2.boxPoints(((center[0], center[1]), (width, height), angle))
    box = np.int0(box)  # Convert to integer
    cv2.drawContours(image, [box], 0, color, thickness)
    cv2.circle(image, (int(round(center[0])), int(round(center[1]))), radius=2, color=color, thickness=-1)

In [72]:
# Function to crop a video
def crop_video(file_path, center, width, height, angle = 0):

    folder_path = os.path.dirname(file_path)
    output_folder = os.path.join(folder_path, "cropped")
    os.makedirs(output_folder, exist_ok=True)
    output_path = os.path.join(output_folder, os.path.basename(file_path))
    
    cap = cv2.VideoCapture(file_path)
    if not cap.isOpened():
        print(f"Error: Cannot open {file_path}.")
        return
        
    # Output video writer with the cropped size
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS), (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        if angle != 0:
            # Get the rotation matrix
            rotation_matrix = cv2.getRotationMatrix2D(tuple(center), angle, 1.0)

            # Rotate the entire frame
            frame = cv2.warpAffine(frame, rotation_matrix, (frame.shape[1], frame.shape[0]))
        
        
        # Crop the rotated region
        frame = frame[
            center[1] - height // 2:center[1] + height // 2,
            center[0] - width // 2:center[0] + width // 2,
        ]

        out.write(frame)

    cap.release()
    out.release()
    print(f"Cropped video saved to {output_path} with size {width}x{height}.")

---
#### Lets start with "cropper"
This function will let you crop the video to the desired size.
It's useful for when you have unused space in the video, or if you record multiple animals simultaneously with one camera.

In [73]:
def cropper():

    # Get video files
    video_files = select_video_files()

    # Merge frames
    image = merge_frames(video_files)
    
    # Get original dimensions and print them
    width = image.shape[1]
    height = image.shape[0]
    print(f"Original Size: {width}x{height}")

    # Initialize variables
    clone = image.copy()
    corners = []  # Current ROI 
    dragging = [False]  # For moving ROI
    drag_start = None
    angle = 0  # Current angle for rotation
    rotate_factor = 1  # Amount of change per scroll
    resize_factor = 2  # Amount of change per scroll

    # Mouse callback function
    def handle_mouse(event, x, y, flags, param):
        nonlocal clone, corners, dragging,drag_start, angle, rotate_factor, resize_factor

        # Start drawing the rectangle
        if event == cv2.EVENT_LBUTTONDOWN:
            # angle = 0 # Reset angle
            if not dragging[0]:
                dragging[0] = True
                corners = [(x, y)]  # Start new ROI at the clicked position

        # Update rectangle during drawing
        elif event == cv2.EVENT_MOUSEMOVE and dragging[0] and len(corners) == 1:
            x1, y1 = corners[0]
            x2, y2 = x, y # While dragging, update the end point
            center, width, height = define_rectangle(x1, y1, x2, y2)
            clone = image.copy()
            draw_rectangle(clone, center, width, height, angle, (0, 255, 255), 2)
            
        # Finish drawing the rectangle
        elif event == cv2.EVENT_LBUTTONUP:
            if dragging[0]:
                dragging[0] = False
                corners.append((x, y))  # End point

        # Start moving the rectangle
        elif event == cv2.EVENT_RBUTTONDOWN and len(corners) == 2:
            dragging[0] = True
            drag_start = (x, y)

        # Move the rectangle
        elif event == cv2.EVENT_MOUSEMOVE and dragging[0] and len(corners) == 2:
            dx = x - drag_start[0]
            dy = y - drag_start[1]
            drag_start = (x, y)
            x1, y1, x2, y2 = (corners[0][0] + dx, corners[0][1] + dy, corners[1][0] + dx, corners[1][1] + dy)
            corners[0] = x1, y1
            corners[1] = x2, y2
            center, width, height = define_rectangle(x1, y1, x2, y2)
            clone = image.copy()
            draw_rectangle(clone, center, width, height, angle, (0, 255, 255), 2)

        # Stop moving the rectangle
        elif event == cv2.EVENT_RBUTTONUP and len(corners) == 2:
            dragging[0] = False

        # Resize or rotate the ROI using scroll wheel
        elif event == cv2.EVENT_MOUSEWHEEL and len(corners) == 2:
            x1, y1 = corners[0]
            x2, y2 = corners[1]
            if flags & cv2.EVENT_FLAG_CTRLKEY:  # Rotate with Ctrl key pressed
                if flags > 0:  # Scroll up
                    angle -= rotate_factor
                else:  # Scroll down
                    angle += rotate_factor
            else:  # Resize without modifier key
                width = max(abs(x2 - x1), 1)
                height = max(abs(y2 - y1), 1)
                ratio = width/height
                if flags > 0:  # Scroll up
                    x1 -= resize_factor*ratio
                    y1 -= resize_factor
                    x2 += resize_factor*ratio
                    y2 += resize_factor
                else:  # Scroll down
                    x1 += resize_factor*ratio
                    y1 += resize_factor
                    x2 -= resize_factor*ratio
                    y2 -= resize_factor
                corners = [(x1, y1), (x2, y2)]
            center, width, height = define_rectangle(x1, y1, x2, y2)
            clone = image.copy()
            draw_rectangle(clone, center, width, height, angle, (0, 255, 255), 2)

        # Draw the updated ROI and display width, height, and angle
        if len(corners) > 0:
            x1, y1 = corners[0]
            if len(corners) > 1:
                x2, y2 = corners[1]
            else:
                x2, y2 = x, y     
            
            # Display height, width, and angle at the bottom of the frame
            center, width, height = define_rectangle(x1, y1, x2, y2)
            text = f"M: [{x}, {y}], C: {center}, W: {width}, H: {height}, A: {angle}"  # Convert to int for display
        else:
            text = f"M: [{x}, {y}]"

        font_scale, font_thickness = 0.5, 1
        text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness)[0]
        text_x, text_y = 10, clone.shape[0] - 100
        cv2.rectangle(clone, (text_x - 5, text_y - text_size[1] - 5), 
                    (text_x + text_size[0] + 5, text_y + 5), (0, 0, 0), -1)  # Background for text
        cv2.putText(clone, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 
                    font_scale, (255, 255, 255), font_thickness)

    # Set up the OpenCV window and bind the mouse callback
    cv2.namedWindow("Select Region")
    cv2.setMouseCallback("Select Region", handle_mouse)

    while True:
        cv2.imshow("Select Region", clone)
        key = cv2.waitKey(1) & 0xFF

        if key == ord('c') and len(corners) == 2:  # Save the ROI
            response = messagebox.askquestion("Crop", "Do you want to crop this region?")
            if response == 'yes':
                break

        elif key == ord('q'):  # Quit and save
            response = messagebox.askquestion("Exit", "Do you want to exit the cropper?")
            if response == 'yes':
                print("Cropping canceled.")
                cv2.destroyAllWindows()
                return                

    cv2.destroyAllWindows()

    # Ensure valid ROI
    x1, y1 = corners[0]
    x2, y2 = corners[1]
    center, width, height = define_rectangle(x1, y1, x2, y2)

    # Apply cropping
    for file in video_files:
        crop_video(file, center, width, height, angle)

    print(f"Video cropped and saved")

# Call the main function
cropper()

Selected 23 videos.
Original Size: 1920x1080


C:\Users\dhers\AppData\Local\Temp\ipykernel_16848\748426391.py:9: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.int0(box)  # Convert to integer


Cropped video saved to C:/Users/dhers/Desktop/Guille/Exp_12-2024/E+M\cropped\Guille_R01_C1i.avi.
Cropped video saved to C:/Users/dhers/Desktop/Guille/Exp_12-2024/E+M\cropped\Guille_R02_C1d.avi.
Cropped video saved to C:/Users/dhers/Desktop/Guille/Exp_12-2024/E+M\cropped\Guille_R03_C1a.avi.
Cropped video saved to C:/Users/dhers/Desktop/Guille/Exp_12-2024/E+M\cropped\Guille_R04_C2n.avi.
Cropped video saved to C:/Users/dhers/Desktop/Guille/Exp_12-2024/E+M\cropped\Guille_R05_C2i.avi.
Cropped video saved to C:/Users/dhers/Desktop/Guille/Exp_12-2024/E+M\cropped\Guille_R06_C2d.avi.
Cropped video saved to C:/Users/dhers/Desktop/Guille/Exp_12-2024/E+M\cropped\Guille_R07_C3i.avi.
Cropped video saved to C:/Users/dhers/Desktop/Guille/Exp_12-2024/E+M\cropped\Guille_R08_C3d.avi.
Cropped video saved to C:/Users/dhers/Desktop/Guille/Exp_12-2024/E+M\cropped\Guille_R09_C3a.avi.
Cropped video saved to C:/Users/dhers/Desktop/Guille/Exp_12-2024/E+M\cropped\Guille_R10_C4i.avi.
Cropped video saved to C:/User

---
#### It is also useful to resize the video if it is too big or too small.

In [ ]:
def change_video_resolution(input_video_path, output_video_path, width, height):
    """
    Changes the resolution of a video.
    
    Parameters:
    - input_video_path: Path to the input video file.
    - output_video_path: Path to save the resized video.
    - width: Target width of the video.
    - height: Target height of the video.
    """
    # Open the video file
    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        print("Error: Cannot open the video file.")
        return

    # Get the original video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))

    # Define the codec and create a VideoWriter object
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Resize the frame
        resized_frame = cv2.resize(frame, (width, height))

        # Write the frame to the output video
        out.write(resized_frame)

    # Release resources
    cap.release()
    out.release()
    cv2.destroyAllWindows()

    print(f"Video saved to {output_video_path} with resolution {width}x{height}")

# Example usage
change_video_resolution("input.mp4", "output.mp4", 640, 360)
